In [30]:
import pandas as pd
import numpy as np
from astropy.time import Time
from math import radians, cos, sin, asin, sqrt

In [31]:
def haversine(lon1, lat1, lon2, lat2):  #Calculate the great circle distance between two points on the earth 
                                        #(specified in decimal degrees)
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2]) # convert decimal degrees to radians 
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles.
    return c * r

In [84]:
# File of locations of all satellites. Location points per five minutes.
Location_File_All_Sats = "../../Data/Location/ISS_FERMI_Light-1_Locations.csv"
Locations_data_pd = pd.read_csv(Location_File_All_Sats)
Locations_headers = list(Locations_data_pd.keys())
Locations_data    = [i for i in Locations_data_pd.to_numpy().T]
Locations_data    = dict(zip(Locations_headers,Locations_data))

# Distance between all sats using great circle distance:
distance_ISS_Light1 = [haversine(Locations_data['Lon(deg)Light-1'][i],Locations_data['Lat(deg)Light-1'][i],Locations_data['Lon(deg)ISS'][i],Locations_data['Lat(deg)ISS'][i]) for i in range(len(Locations_data['Time(ModJDate)']))]
distance_ISS_Light1 = np.array(distance_ISS_Light1).T
distance_FERMI_Light1 = [haversine(Locations_data['Lon(deg)Light-1'][i],Locations_data['Lat(deg)Light-1'][i],Locations_data['Lon(deg)FERMI'][i],Locations_data['Lat(deg)FERMI'][i]) for i in range(len(Locations_data['Time(ModJDate)']))]
distance_FERMI_Light1 = np.array(distance_FERMI_Light1).T

# Add distances between sats to dict:
Locations_data.update({"ISS_Light-1_Distance(km)": distance_ISS_Light1})
Locations_data.update({"FERMI_Light-1_Distance(km)": distance_FERMI_Light1})

In [86]:
# Filter data line index and time by distance from Light-1:
THRESHOLD_km   = 1000     # Distance threshold (km)

ISS_idx_cut     = np.where([i <= THRESHOLD_km for i in distance_ISS_Light1])[0]
ISS_time_cut    = Locations_data['Time(ModJDate)'][ISS_idx_cut]

FERMI_idx_cut     = np.where([i <= THRESHOLD_km for i in distance_FERMI_Light1])[0]
FERMI_time_cut    = Locations_data['Time(ModJDate)'][FERMI_idx_cut]

# Format time in MJD and ISOT:
ISS_time = [Time(i,format="mjd") for i in ISS_time_cut]
ISS_time = [(i.mjd,i.isot) for i in ISS_time]

FERMI_time = [Time(i,format="mjd") for i in FERMI_time_cut]
FERMI_time = [(i.mjd,i.isot) for i in FERMI_time]

In [88]:
# In MJD: .00300 is 5 minutes.. so 0.01 for safety. 
# This is the threshold that distinguishes two satelite orbit overlaping durations from each other.
TIME_THRESHOLD = 0.01

num = 0 #for time pronted out: 0 for MJD, 1 for UTC
ISS_time_list = [('Start here:', ISS_time[0][num])]
ISS_val = 0 # To decide if correlation started at this time or ended at this time.
for i in range(len(ISS_time)-1):
    if ((ISS_time[i+1][0] - ISS_time[i][0]) >= TIME_THRESHOLD): #must stay in MJD for calculation

        if ((ISS_val % 2) == 0): # If val is even or odd
            k = ("End here:", ISS_time[i][num])
            
        else:
            k = ("Start here:", ISS_time[i+1][num])
        
        ISS_time_list.append(k)
        ISS_val = ISS_val + 1
k = ("End here:", ISS_time[-1][num])
ISS_time_list.append(k)


FERMI_time_list = [('Start here:', FERMI_time[0][num])]
FERMI_val = 0 # To decide if start or end
for i in range(len(FERMI_time)-1):
    if ((FERMI_time[i+1][0] - FERMI_time[i][0]) >= TIME_THRESHOLD):

        if ((FERMI_val % 2) == 0): # If val is even or odd
            k = ("End here:", FERMI_time[i][num])
            
        else:
            k = ("Start here:", FERMI_time[i+1][num])
        
        FERMI_time_list.append(k)
        FERMI_val = FERMI_val + 1
k = ("End here:", FERMI_time[-1][num])
FERMI_time_list.append(k)

In [47]:
# ISS_time_list
# FERMI_time_list

In [89]:
FERMI_Data_During_Light1_Mission = "../../Data/Other_Sat_Data/FERMI_all_Data_During_Light-1_Mission.csv"

FERMI_data_pd = pd.read_csv(FERMI_Data_During_Light1_Mission)
FERMI_Headers = list(FERMI_data_pd.keys())
FERMI_data    = [i for i in FERMI_data_pd.to_numpy().T]
FERMI_data    = dict(zip(FERMI_Headers,FERMI_data))

In [90]:
# FERMI_time = [Time(i,format="mjd") for i in FERMI_time_cut]
# FERMI_time = [(i.mjd,i.isot) for i in FERMI_time]
new_time_threshold = 0.003
for i in range(len(FERMI_time_list)-1):
    if 'Start' in FERMI_time_list[i][0] and 'End' in FERMI_time_list[i+1][0]:
        for k in range(len(FERMI_data['Trigger_Time(MJD)'])-1):
            # print (k)
            if  (FERMI_time_list[i][1] - new_time_threshold) < FERMI_data['Trigger_Time(MJD)'][k] < (FERMI_time_list[i+1][1] + new_time_threshold):
                
                Print_line = (FERMI_data['Trigger_Time(MJD)'][k],FERMI_data['Name'][k],FERMI_data['Trigger_Timescale(ms)'][k])
                print (Print_line)

# now add distance

(59892.6063671, 'SGR221109606 ', 16)


In [ ]:
# maybe corrilate FERMI data with ASIM later (LIKE WE HAVE IT!!)